## Point Data Extraction from the AI4SH Datacube

Latest update: 7 December 2025

### Overview
This Jupyter Notebook extracts static environmental values from the AI4SoilHealth (AI4SH) Datacube for specified in-situ point locations. It reads coordinate points from an Excel file, samples raster layers hosted on the EcoDataCube S3 server, and exports the results to a CSV file.

### Purpose
The notebook enables researchers to:
- Load AI4SH in-situ coordinate points from an Excel file
- Convert point data to a GeoDataFrame with proper coordinate reference systems
- Extract values from multiple static raster values (terrain derivatives, crop types, etc.)
- Handle coordinate reprojection automatically when needed
- Save the enriched point data with extracted values

### Workflow
1. **Environment Setup**: Load required Python libraries (pandas, geopandas, rasterio, numpy)
2. **Data Loading**: Read in-situ coordinate points from Excel file
3. **Spatial Data Preparation**: Convert points to GeoDataFrame with EPSG:4326 CRS
4. **value Selection**: Define list of raster layer URLs from AI4SH Datacube
5. **Value Extraction**: Loop through each layer and sample values at point locations
6. **Data Export**: Save results to CSV file with all extracted values

### Python Package Requirements

The Notebook requires the following third party Python packages:
- pandas 
- geopandas 
- rasterio 
- numpy 
- openpyxl 
- gdal

The terminal command for installing these packages with [Anaconda](https://www.anaconda.com) is given below.

### Input Requirements
- **Excel file**: `AI4SH_in-situ_coordinate_points.xlsx` containing columns for `longitude` and `latitude`
- **Directory structure**: Excel file should be located in `../AI4SH_point_locations/` relative to notebook

### Output
- **CSV file**: `AI4SH_in-situ_points_with_static_values.csv` saved to `../AI4SH_point_data/`
- Contains original point data plus columns for each extracted value

### Data Sources
Static values are accessed from the [AI4SoilHealth SoilHealthDataCube](https://github.com/AI4SoilHealth/SoilHealthDataCube) via HTTPS, including:
- Terrain derivatives (slope, curvature, hillshade, TWI, etc.)
- Geomorphological features (geomorphons, openness indices)
- Topographic indices (LS-factor, shape index)
- Land cover/crop type data

### Dependencies
- Python 3.12
- pandas: Data manipulation and Excel/CSV I/O
- geopandas: Spatial data operations and coordinate transformations
- rasterio: Raster data reading and sampling
- numpy: Numerical operations and handling missing values
- openpyxl: Excel file reading support

### Notes
- The notebook handles coordinate reprojection automatically when raster CRS differs from point CRS
- NoData values are converted to NaN for proper handling in pandas
- Progress messages indicate which value is currently being processed
- Internet connection required to access remote raster data from S3 server

### Install Python environment
Before you run the Notebook you must create a python environment with the required python packages. One way to do that is to install the virtual python environment manager [Anaconda](https://www.anaconda.com/docs/getting-started/anaconda/install). With Anaoconda installed, run the following command from a terminal window to create a virtual python environment (_ai4sh_datacube_access_312_):
```
conda create -n ai4sh_datacube_access_312 -c conda-forge  pandas geopandas rasterio numpy openpyxl gdal python=3.12 
```

### Iinitiate the Notebook
Start the notebook and choose your python environment (_ai4sh_datacube_access_312_). You might also be asked to install the additional package **ipykernel** for running the Notebook - the Notebook will pop-up a window and ask for it if required. Aaccept and install the package. Once that is finished the Notebook should work.

In [1]:
# Load AI4SH in-situ coordinate points dataset and convert to GeoDataFrame

# Standard library imports
import os

# Third parthy imports
import pandas as pd

import geopandas as gpd

# Get the current working directory
notebook_path = os.getcwd()

# Get the repository path by going up one level from the notebook path
repo_path = os.path.abspath(os.path.join(notebook_path, '..'))

# Construct the local path to the directory with the AI4SH in-situ coordinate points
point_location_path = os.path.join(repo_path, 'AI4SH_point_locations')

# Change the current operating system working directory to the local path with the AI4SH in-situ coordinate points
os.chdir(point_location_path)  

# Set the file name of the excel file with the AI4SH in-situ coordinate points 
coordinate_points_excel_file_name = 'AI4SH_in-situ_coordinate_points.xlsx'  

# Load the AI4SH in-situ coordinate points dataset
AI4SH_points = pd.read_excel(coordinate_points_excel_file_name)

# Convert all strings to lowercase to ensure consistency
AI4SH_points = AI4SH_points.applymap(lambda s: s.lower() if type(s) == str else s)

# Display the first few rows of the coordinate points dataset
print(AI4SH_points.head())

# Create a GeoDataFrame
geometry = gpd.points_from_xy(AI4SH_points['longitude'], AI4SH_points['latitude'])

# print the coordinate reference system (CRS) of the geometry
print(geometry.crs)

# Convert the DataFrame to a GeoDataFrame
AI4SH_points = gpd.GeoDataFrame(
    AI4SH_points,
    geometry=geometry,
    crs='EPSG:4326'  # Global geographic coordinate system (lat/lon)
)

# Print the CRS of the GeoDataFrame
print(AI4SH_points.crs)

# When this cell finishes running you should see the first few rows of 
# the AI4SH in-situ coordinate points dataset and the coordinate 
# reference system (CRS) of the geometry and GeoDataFrame printed out.

  pilot_country  pilot_site   point_id   latitude  longitude
0            se  loennstorp     1-sand  55.658400  13.058864
1            se  loennstorp     2-sand  55.658636  13.060314
2            se  loennstorp  3-organic  55.650342  13.079697
3            se  loennstorp  4-organic  55.649533  13.079633
4            se  loennstorp  5-organic  55.651619  13.082658
None
EPSG:4326


/var/folders/6s/8smwkls52tb_d8xth6pgqdx80000gn/T/ipykernel_35466/509983434.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  AI4SH_points = AI4SH_points.applymap(lambda s: s.lower() if type(s) == str else s)


### List the static values you want to access
See the [AI4SoilHealth SoilHealthDataCube](https://github.com/AI4SoilHealth/SoilHealthDataCube) documentation for availability of both static and time varying data that is available. The datacube is regularly updated and more data is continuously being added.

In [2]:
# Define the list of value URLs to rtrieve data from
value_urls = [
    "https://s3.ecodatacube.eu/arco/dfme_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/geomorphon_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/hillshade_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/ls.factor_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/maxic_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/minic_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/neg.openness_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/pos.openness_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/pro.curv_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/shpindx_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/slope.in.degree_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/twi_edtm_m_30m_s_20000101_20221231_eu_epsg.3035_v20241230.tif",
    "https://s3.ecodatacube.eu/arco/crop.type_eucropmap.v1_c_10m_s_20220101_20221231_eu_epsg.3035_v20250416.tif"
]

### Extract the point values from static raster layers
Connect to the [AI4SoilHealth SoilHealthDataCube](https://github.com/AI4SoilHealth/SoilHealthDataCube) and sequentially open each dataset (covaraite URL) and loop over the points defined in excel file to retrieve the values.

In [3]:
# Extract values at point locations

# Third parthy imports
import rasterio

import numpy as np

# Create a copy of the points GeoDataFrame to store results
results = AI4SH_points.copy()

# Loop through each value URL and extract values at point locations
for url in value_urls:

    # Extract value name from URL
    value_name = url.split("/")[-1].split(".tif")[0]

    # Extract the first part of the coviariate name as a simplified name 
    parts = value_name.split("_")

    simplified_name = "_".join(parts[:2])

    # Print out the simplified value name being processed
    print(f"Processing: {simplified_name}")

    # Open the raster file for the value that is extracted
    with rasterio.open(url) as src:

        # Reproject points if needed
        if AI4SH_points.crs != src.crs:

            points_proj = AI4SH_points.to_crs(src.crs)

        else:

            points_proj = AI4SH_points

        # Prepare list of (x, y) coordinates
        coords = [(point.x, point.y) for point in points_proj.geometry]

        # Sample raster at points
        values = []

        # Loop through the coordinates listed in the excel file and extract the values
        for val in src.sample(coords):

            v = val[0]

            if v == src.nodata or v is None:

                v = np.nan

            values.append(v)

        # Add the extracted values to the results GeoDataFrame
        results[simplified_name] = values

# Display the first few rows of the results GeoDataFrame
print(results.head())

Processing: dfme_edtm
Processing: geomorphon_edtm
Processing: geomorphon_edtm
Processing: hillshade_edtm
Processing: hillshade_edtm
Processing: ls.factor_edtm
Processing: ls.factor_edtm
Processing: maxic_edtm
Processing: maxic_edtm
Processing: minic_edtm
Processing: minic_edtm
Processing: neg.openness_edtm
Processing: neg.openness_edtm
Processing: pos.openness_edtm
Processing: pos.openness_edtm
Processing: pro.curv_edtm
Processing: pro.curv_edtm
Processing: shpindx_edtm
Processing: shpindx_edtm
Processing: slope.in.degree_edtm
Processing: slope.in.degree_edtm
Processing: twi_edtm
Processing: twi_edtm
Processing: crop.type_eucropmap.v1
Processing: crop.type_eucropmap.v1
  pilot_country  pilot_site sample_id   latitude  longitude  \
0            SE  Loennstorp   1-sandy  55.658400  13.058864   
1            SE  Loennstorp   2-sandy  55.658636  13.060314   
2            SE  Loennstorp     3-org  55.650342  13.079697   
3            SE  Loennstorp     4-org  55.649533  13.079633   
4      

### Save the extracted data
Set the file path and name where you want to save the extracted data as a **.csv** file. By defualt the path is direclty under the root of the repositiry and thus a sibling to the folders _notebook_ and _AI4SH_point_locations_ with the name _AI4SH_point_data_, and the default file bane is _AI4SH_in-situ_points_with_static_values.csv_. By editing the puython block below you can change both the path and the filename for svaing the results.

In [4]:
# Save the extracted data to a CSV file

# Construct the local path to the directory where you want to save the results
point_data_path = os.path.join(repo_path, 'AI4SH_point_data')

# Create the directory if it doesn't exist
if not os.path.exists(point_data_path):

    os.makedirs(point_data_path)

# Set the output file path
AI4SH_datacube_insitu_point_data_filename = 'AI4SH_in-situ_points_with_static_values.csv'

output_file = os.path.join(point_data_path, AI4SH_datacube_insitu_point_data_filename)

results.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")

Results saved to /Users/thomasgumbricht/GitHub_AI4SH/EO-data_access/AI4SH_point_data/AI4SH_in-situ_points_with_static_values.csv
